In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:
--SONARCLOUD_COMPONENTS

-- CREATE OR REPLACE TABLE prod.silver.SONARCLOUD_COMPONENTS (
--     ORGANIZATION VARCHAR(16777216),
--     KEY VARCHAR(16777216),
--     NAME VARCHAR(16777216),
--     QUALIFIER VARCHAR(16777216),
--     PROJECT VARCHAR(16777216),
--     timestamp timestamp_tz
-- );

truncate table prod.silver.SONARCLOUD_COMPONENTS;

insert into  prod.silver.SONARCLOUD_COMPONENTS
select
    ORGANIZATION,
    KEY,
    NAME,
    QUALIFIER,
    PROJECT,
    current_timestamp as timestamp
from prod.raw.sonarcloud_components;






In [ ]:

-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- SONARCLOUD_PROJECTS

-- CREATE OR REPLACE TABLE prod.silver.SONARCLOUD_PROJECTS (
--     KEY VARCHAR(16777216),
--     NAME VARCHAR(16777216),
--     REVISION VARCHAR(16777216),
--     -- QUALIFIER VARCHAR(16777216),
--     -- VISIBILITY VARCHAR(16777216),
--     -- ORGANIZATION VARCHAR(16777216),
--     LASTANALYSISDATE timestamp_tz,
--     timestamp timestamp_tz
-- );

truncate table prod.silver.SONARCLOUD_PROJECTS;

insert into prod.silver.SONARCLOUD_PROJECTS
select
    "KEY",
    "NAME",
    "REVISION",
to_timestamp_tz(
            REGEXP_REPLACE(LASTANALYSISDATE, '(\\d{2})(\\d{2})$', '\\1:\\2')
            ) AS "LASTANALYSISDATE",
    current_timestamp as timestamp
from prod.raw.SONARCLOUD_PROJECTS;

In [ ]:
select * from prod.silver.SONARCLOUD_PROJECTS

In [ ]:

-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- SONARCLOUD_METRICS

-- CREATE OR REPLACE TABLE prod.silver.SONARCLOUD_METRICS
--     (
--     KEY VARCHAR(16777216),
--     NAME VARCHAR(16777216),
--     TYPE VARCHAR(16777216),
--     DOMAIN VARCHAR(16777216),
--     HIDDEN	BOOLEAN,
--     DIRECTION	FLOAT,
--     DESCRIPTION	VARCHAR(16777216),
--     QUALITATIVE	BOOLEAN,
--     DECIMALSCALE FLOAT,
--     timestamp timestamp_tz
--     );

truncate table prod.silver.SONARCLOUD_METRICS;

insert into prod.silver.SONARCLOUD_METRICS
select
    KEY,
    NAME,
    TYPE,
    DOMAIN,
    HIDDEN,
    DIRECTION,
    DESCRIPTION,
    QUALITATIVE,
    DECIMALSCALE
    ,current_timestamp as timestamp
from prod.raw.SONARCLOUD_METRICS;

In [ ]:
-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --
-- SONARCLOUD_ISSUES


-- create or replace table prod.silver.SONARCLOUD_ISSUES
-- (
-- issue_key VARCHAR(16777216),
-- DEBT	VARCHAR(16777216),
-- HASH	VARCHAR(16777216),
-- LINE	FLOAT,
-- RULE	VARCHAR(16777216),
-- TAGS	ARRAY,
-- TYPE	VARCHAR(16777216),
-- FLOWS	ARRAY,
-- AUTHOR	VARCHAR(16777216),
-- EFFORT	VARCHAR(16777216),
-- STATUS	VARCHAR(16777216),
-- IMPACTS	ARRAY,
-- MESSAGE	VARCHAR(16777216),
-- PROJECT	VARCHAR(16777216),
-- ASSIGNEE	VARCHAR(16777216),
-- SEVERITY	VARCHAR(16777216),
-- CLOSEDATE	VARCHAR(16777216),
-- COMPONENT	VARCHAR(16777216),
-- TEXTRANGE	OBJECT,
-- RESOLUTION	VARCHAR(16777216),
-- UPDATEDATE	VARCHAR(16777216),
-- ISSUESTATUS	VARCHAR(16777216),
-- CREATIONDATE	VARCHAR(16777216),
-- ORGANIZATION	VARCHAR(16777216),
-- CLEANCODEATTRIBUTE	VARCHAR(16777216),
-- EXTERNALRULEENGINE	VARCHAR(16777216),
-- CLEANCODEATTRIBUTECATEGORY	VARCHAR(16777216),
-- impact_severity string,
-- software_quality string,
-- timestamp timestamp_tz
-- );

truncate table prod.silver.SONARCLOUD_ISSUES ;

INSERT INTO prod.silver.SONARCLOUD_ISSUES (
    issue_key,
    TYPE,
    AUTHOR,
    MESSAGE,
    LINE,
    ISSUESTATUS,
    EFFORT,
    UPDATEDATE,
    RULE,
    PROJECT,
    TAGS,
    CLEANCODEATTRIBUTECATEGORY,
    STATUS,
    CREATIONDATE,
    SEVERITY,
    EXTERNALRULEENGINE,
    RESOLUTION,
    ASSIGNEE,
    CLOSEDATE,
    COMPONENT,
    impact_severity,
    software_quality,
    timestamp
)
SELECT 
    issues.KEY AS issue_key,
    issues.TYPE,
    issues.AUTHOR,
    issues.MESSAGE,
    issues.LINE,
    issues.ISSUESTATUS,
    issues.EFFORT,
    issues.UPDATEDATE,
    issues.RULE,
    issues.PROJECT,
    issues.TAGS,
    issues.CLEANCODEATTRIBUTECATEGORY,
    issues.STATUS,
    issues.CREATIONDATE,
    issues.SEVERITY,
    issues.EXTERNALRULEENGINE,
    issues.RESOLUTION,
    issues.ASSIGNEE,
    issues.CLOSEDATE,
    issues.COMPONENT,
    impacts.VALUE:"severity"::STRING AS impact_severity,
    impacts.VALUE:"softwareQuality"::STRING AS software_quality,
    current_timestamp() AS timestamp
FROM prod.raw.SONARCLOUD_ISSUES AS issues,
LATERAL FLATTEN(input => issues.IMPACTS) AS impacts
WHERE
    issues.KEY IS NOT NULL
    AND issues.TYPE IS NOT NULL
    AND issues.PROJECT IS NOT NULL
    AND issues.STATUS IS NOT NULL
    AND issues.UPDATEDATE IS NOT NULL
    AND issues.SEVERITY IS NOT NULL;

In [ ]:

-- -- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --
-- SONARCLOUD_MEASURES_COMPONENT

-- CREATE OR REPLACE TABLE prod.silver.SONARCLOUD_MEASURES_COMPONENT (
--   id STRING,
--   key STRING,
--   name STRING,
--   files INTEGER,
--   codeLines INTEGER,
--   maintainability_total INTEGER,
--   maintainability_high INTEGER,
--   maintainability_medium INTEGER,
--   maintainability_low INTEGER,
--   reliability_total INTEGER,
--   reliability_high INTEGER,
--   reliability_medium INTEGER,
--   reliability_low INTEGER,
--   security_total INTEGER,
--   security_high INTEGER,
--   security_medium INTEGER,
--   security_low INTEGER,
--   securityHotspots INTEGER,
--   lineCoverage INTEGER,
--   duplicated_lines INTEGER,
--   duplicated_lines_density FLOAT,
--   timestamp TIMESTAMP_TZ
-- );

TRUNCATE TABLE prod.silver.SONARCLOUD_MEASURES_COMPONENT;

INSERT INTO prod.silver.SONARCLOUD_MEASURES_COMPONENT (
  id, key, name, files, codeLines, maintainability_total, maintainability_high,
  maintainability_medium, maintainability_low, reliability_total, reliability_high,
  reliability_medium, reliability_low, security_total, security_high, security_medium,
  security_low, securityHotspots, lineCoverage, duplicated_lines,
  duplicated_lines_density, timestamp
)
WITH PIVOTED_TABLE AS (
  SELECT *
  FROM (
    SELECT
      t.ID AS id,
      t.KEY AS key,
      t.NAME AS name,
      m.value:metric::STRING AS metric,
      m.value:value::STRING AS value
    FROM prod.raw.SONARCLOUD_MEASURES_COMPONENT t,
    LATERAL FLATTEN(input => t.MEASURES) m
  )
  PIVOT (
    MAX(VALUE) FOR METRIC IN (
      'files', 'ncloc', 'maintainability_issues', 'reliability_issues', 
      'security_hotspots', 'security_issues', 'line_coverage', 
      'duplicated_lines', 'duplicated_lines_density'
    )
  )
)
SELECT
  id,
  key,
  name,
  TRY_CAST(w."'files'" AS INTEGER) AS files,
  TRY_CAST(w."'ncloc'" AS INTEGER) AS codeLines,
  TRY_CAST(PARSE_JSON(w."'maintainability_issues'"):total::STRING AS INTEGER) AS maintainability_total,
  TRY_CAST(PARSE_JSON(w."'maintainability_issues'"):HIGH::STRING AS INTEGER) AS maintainability_high,
  TRY_CAST(PARSE_JSON(w."'maintainability_issues'"):MEDIUM::STRING AS INTEGER) AS maintainability_medium,
  TRY_CAST(PARSE_JSON(w."'maintainability_issues'"):LOW::STRING AS INTEGER) AS maintainability_low,
  TRY_CAST(PARSE_JSON(w."'reliability_issues'"):total::STRING AS INTEGER) AS reliability_total,
  TRY_CAST(PARSE_JSON(w."'reliability_issues'"):HIGH::STRING AS INTEGER) AS reliability_high,
  TRY_CAST(PARSE_JSON(w."'reliability_issues'"):MEDIUM::STRING AS INTEGER) AS reliability_medium,
  TRY_CAST(PARSE_JSON(w."'reliability_issues'"):LOW::STRING AS INTEGER) AS reliability_low,
  TRY_CAST(PARSE_JSON(w."'security_issues'"):total::STRING AS INTEGER) AS security_total,
  TRY_CAST(PARSE_JSON(w."'security_issues'"):HIGH::STRING AS INTEGER) AS security_high,
  TRY_CAST(PARSE_JSON(w."'security_issues'"):MEDIUM::STRING AS INTEGER) AS security_medium,
  TRY_CAST(PARSE_JSON(w."'security_issues'"):LOW::STRING AS INTEGER) AS security_low,
  TRY_CAST(w."'security_hotspots'" AS INTEGER) AS securityHotspots,
  TRY_CAST(w."'line_coverage'" AS INTEGER) AS lineCoverage,
  TRY_CAST(w."'duplicated_lines'" AS INTEGER) AS duplicated_lines,
  TRY_CAST(w."'duplicated_lines_density'" AS FLOAT) AS duplicated_lines_density,
  current_timestamp() as timestamp
FROM PIVOTED_TABLE w;